这是一个使用CodeFuse-Query分析Go项目的教程。在教程中，你将体验到使用命令行工具对代码仓库进行数据化，然后使用Godel语言来分析这个仓库。

环境准备：配置cli的路径，你可以将sparrow cli包导出到系统PATH，如 `!export PATH=/Users/cyz/work/opensource/cli:$PATH`：

In [3]:
!export PATH=/Users/cyz/work/opensource/cli:$PATH

export PATH=/Users/cyz/work/opensource/cli:$PATH



检查cli是否就绪

In [10]:
!which sparrow

which sparrow
/Users/cyz/work/opensource/cli/sparrow



STEP 0: 克隆要分析的仓库。我们以 [gorm](https://github.com/go-gorm/gorm.git) 项目为例。

In [ ]:
!git clone https://github.com/go-gorm/gorm.git

STEP 1: 代码数据化。使用 `sparrow database create` 命令创建一个db文件，指定待分析的仓库地址（当前目录下的gorm子目录），分析的语言（go），以及db文件的存储路径（放置在当前目录下的/db/gorm）。执行该命令之后，竟会生成一份db文件，该文件存储着代码仓库的结构化数据，之后的分析就是针对这份数据进行。

In [11]:
!sparrow database create --source-root gorm --data-language-type go --output ./db/gorm

sparrow database create --source-root gorm --data-language-typ

<base create --source-root gorm --data-language-type                         go --output ./db/gorm


STEP 2: 使用Godel分析语言分析db文件。在本教程中，可以点击代码左侧的执行按钮，直接运行分析脚本。在命令行中，你可以使用 `sparrow query run` 命令来执行查询脚本，具体可以使用`sparrow query run -h` 来获取详细参数信息。

<b>示例</b> 查询 [gorm](https://github.com/go-gorm/gorm.git) 的文件代码复杂度信息。

第一行通过内核魔法命令指定分析的db路径，后面写查询文件代码复杂度Godel脚本。

In [4]:
%db ~/work/opensource/db/gorm
// script
use coref::go::*

fn default_db() -> GoDB {
    return GoDB::load("coref_go_src.db")
}

/**
 * @param name: 文件名
 * @param func: 函数名
 * @param cmplx: 函数圈复杂度
 * @param sl,el,sc,ec: 函数位置信息，依次为函数起始行，结束行
 */
fn out(name: string, func: string, cmplx: int, sl: int, el: int) -> bool {
    for(f in GoFile(default_db()), function in Function(default_db())) {
        if ((!f.isAutoGenereatedFile()) &&
            f.key_eq(function.getBelongsFile()) &&
            name = f.getName() &&
            func = function.getName() &&
            cmplx = function.getCyclomaticComplexity() &&
            sl = function.getLocation().getStartLineNumber() &&
            el = function.getLocation().getEndLineNumber()) {
            return true
        }
    }
}

fn main() {
    output(out())
}

/Users/cyz/work/opensource/db/gorm


Sparrow database is set to: /Users/cyz/work/opensource/db/gorm


<el-jupyter-hz9fci6t --verbose -d /Users/cyz/work/op                        ensource/db/gormyz/work/o
2023-11-23 16:37:52,984 INFO: sparrow bc184bb8 will start
2023-11-23 16:37:52,984 INFO: database /Users/cyz/work/opensource/db/gorm/coref_go_src.db size: 41.99 MB
2023-11-23 16:37:52,986 INFO: execute : /Users/cyz/work/opensource/cli/godel-script/usr/bin/godel /private/var/folders/8r/_7k9xlsn01d967dg0p74wwpc0000gp/T/godel-jupyter-hz9fci6t/query.gdl -p /Users/cyz/work/opensource/cli/lib-1.0 -o /var/folders/8r/_7k9xlsn01d967dg0p74wwpc0000gp/T/tmph51zpaiy.gdl --verbose
[-][ 1.42242 ms] package scan
packages:
coref::cfamily    -> /Users/cyz/work/opensource/cli/lib-1.0/coref.cfamily.gdl
coref::go         -> /Users/cyz/work/opensource/cli/lib-1.0/coref.go.gdl
coref::java       -> /Users/cyz/work/opensource/cli/lib-1.0/coref.java.gdl
coref::javascript -> /Users/cyz/work/opensource/cli/lib-1.0/coref.javascript.gdl
coref::properties -> /Users/cyz/work/opensource/cli/lib-1.0/coref.properties.gdl

,name,func,cmplx,sl,el
0,callbacks.go,Delete,1,63,65
1,callbacks.go,Query,1,55,57
2,callbacks.go,Update,1,59,61
3,callbacks.go,Get,4,155,162
4,callbacks.go,Create,1,51,53
...,...,...,...,...,...
936,tests/upsert_test.go,TestUpsertWithSave,9,138,198
937,tests/upsert_test.go,TestFindOrInitialize,25,200,245
938,tests/upsert_test.go,TestFindOrCreate,30,247,311
939,tests/upsert_test.go,TestUpdateWithMissWhere,3,313,328


保存上一次运行的 query 结果保存到一个JSON文件

In [ ]:
%%save_to ~/work/opensource/out/query.json

STEP 3: 好了，你可以针对分析生成的结果，进行进一步的代码分析了，比如你可以结合pandas库，使用刚刚生成的 query.json 实现最大函数复杂度的排序查询：

In [9]:
%%python
import pandas as pd
data = pd.read_json('~/work/opensource/out/query.json')
data.sort_values('cmplx', ascending=False, inplace=True)
top_10 = data.head(10)
print(top_10)

                      name                       func  cmplx   sl   el
412        schema/field.go                 ParseField    110   99  436
247    callbacks/update.go       ConvertToAssignments     69  131  304
281       schema/schema.go  ParseWithSpecialTableName     68  121  370
221    callbacks/create.go      ConvertToCreateValues     64  217  385
125                scan.go                       Scan     62  125  342
214           statement.go             BuildCondition     61  284  465
413        schema/field.go       setupValuerAndSetter     46  439  969
860  tests/migrate_test.go         TestMigrateColumns     44  517  645
182   migrator/migrator.go              MigrateColumn     41  441  542
269     callbacks/query.go              BuildQuerySQL     41   33  266


Enjoy！